<a href="https://colab.research.google.com/github/derektorquette/processamento-de-lingua-natural-ufmg/blob/main/8_parsing_de_dependencia_com_spacy_e_stanza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Parsing de Dependência**

## Em árvores de dependência, as relações entre as palavras de uma sentença são representadas por meio de relações sintáticas entre elas, tendo uma delas, chamada de raiz/root, como ponto de partida.

 - biblioteca Spacy


## Baixando as dependências

In [ ]:
!pip3 install -U pip setuptools wheel
!pip3 install -U spacy[cuda102]
!pip3 install tabulate

In [ ]:
!python3 -m spacy download pt_core_news_lg # modelo treinado para o Português
!python3 -m spacy download en_core_web_trf  # modelo treinado para o Ingês

In [39]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load('pt_core_news_lg')

## Obtendo as dependências

## Parsing em Português

In [11]:
doc = nlp('Água sanitária pode ser eficaz no combate às larvas do mosquito da dengue.')

In [15]:
doc[0]

Água

In [16]:
for token in doc:
  print(token.lemma_, token.pos_, token.dep_, token.head)

Água NOUN nsubj pode
sanitário ADJ amod Água
poder VERB ROOT pode
ser AUX cop eficaz
eficaz ADJ xcomp pode
em o ADP case combate
combate NOUN obl eficaz
a o ADP case larvas
larva NOUN nmod combate
de o ADP case mosquito
mosquito NOUN nmod larvas
de o ADP case dengue
dengue NOUN nmod mosquito
. PUNCT punct pode


In [20]:
import tabulate

data = []
for token in doc:
  data.append((token.i, token.lemma_, token.pos_, token.morph, token.dep_, token.head))

header = ['Idx', 'Lemma', 'Classe de Palvra', 'Morfologia', 'Dependência', 'Governador']
print(tabulate.tabulate(data, header))

  Idx  Lemma      Classe de Palvra    Morfologia                                             Dependência    Governador
-----  ---------  ------------------  -----------------------------------------------------  -------------  ------------
    0  Água       NOUN                Gender=Fem|Number=Sing                                 nsubj          pode
    1  sanitário  ADJ                 Gender=Fem|Number=Sing                                 amod           Água
    2  poder      VERB                Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin  ROOT           pode
    3  ser        AUX                 VerbForm=Inf                                           cop            eficaz
    4  eficaz     ADJ                 Gender=Fem|Number=Sing                                 xcomp          pode
    5  em o       ADP                 Definite=Def|Gender=Masc|Number=Sing|PronType=Art      case           combate
    6  combate    NOUN                Gender=Masc|Number=Sing                

## buscando subestruturas

In [21]:
def get_subfrase(root, tokens, subfrase):
  subfrase.append((root.i, str(root)))

  for token in root.children:
    subfrase = get_subfrase(token, tokens, subfrase)
  return subfrase

In [22]:
doc[4]

eficaz

In [36]:
r = get_subfrase(doc[10], doc, [])

[w[1] for w in sorted(r, key=lambda x: x[0])]



['do', 'mosquito', 'da', 'dengue']

# **Stanza**

- outro modelo de estado-da-arte para Processamento de Língua Natural

https://stanfordnlp.github.io/stanza/

In [ ]:
!pip3 install stanza

## **Importando a biblioteca e baixando o modelo treinado**

In [ ]:
import stanza
stanza.download('pt')

In [ ]:
nlp = stanza.Pipeline('pt')

In [47]:
doc = nlp("Água sanitária pode ser eficaz no combate às larvas do mosquito da dengue.")

In [ ]:
for snt in doc.sentences:
  print(snt)

In [53]:
for snt in doc.sentences:
    for token in snt.words:
      head = snt.words[token.head-1].text if token.head > 0 else "root"
      print(token.id, token.text, token.upos, token.feats, token.deprel, head)

1 Água NOUN Gender=Fem|Number=Sing nsubj pode
2 sanitária ADJ Gender=Fem|Number=Sing amod Água
3 pode VERB Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin root root
4 ser AUX VerbForm=Inf cop eficaz
5 eficaz ADJ Gender=Fem|Number=Sing xcomp pode
6 em ADP None case combate
7 o DET Definite=Def|Gender=Masc|Number=Sing|PronType=Art det combate
8 combate NOUN Gender=Masc|Number=Sing obl eficaz
9 a ADP None case larvas
10 as DET Definite=Def|Gender=Fem|Number=Plur|PronType=Art det larvas
11 larvas NOUN Gender=Fem|Number=Plur nmod combate
12 de ADP None case mosquito
13 o DET Definite=Def|Gender=Masc|Number=Sing|PronType=Art det mosquito
14 mosquito NOUN Gender=Masc|Number=Sing nmod larvas
15 de ADP None case dengue
16 a DET Definite=Def|Gender=Fem|Number=Sing|PronType=Art det dengue
17 dengue NOUN Gender=Fem|Number=Sing nmod mosquito
18 . PUNCT None punct eficaz


In [55]:
data = []
for snt in doc.sentences:
    for token in snt.words:
      head = snt.words[token.head-1].text if token.head > 0 else "root"
      data.append((token.id, token.text, token.upos, token.feats, token.deprel, head))

header = ['Idx', 'Lemma', 'Classe de Palvra', 'Morfologia', 'Dependência', 'Governador']
print(tabulate.tabulate(data, header))

  Idx  Lemma      Classe de Palvra    Morfologia                                             Dependência    Governador
-----  ---------  ------------------  -----------------------------------------------------  -------------  ------------
    1  Água       NOUN                Gender=Fem|Number=Sing                                 nsubj          pode
    2  sanitária  ADJ                 Gender=Fem|Number=Sing                                 amod           Água
    3  pode       VERB                Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin  root           root
    4  ser        AUX                 VerbForm=Inf                                           cop            eficaz
    5  eficaz     ADJ                 Gender=Fem|Number=Sing                                 xcomp          pode
    6  em         ADP                                                                        case           combate
    7  o          DET                 Definite=Def|Gender=Masc|Number=Sing|Pr